# AI4ER Induction Week - Applied ML session 3
### Designing a machine learning algorithm for predicting extreme weather events in the [ClimateNet](https://gmd.copernicus.org/articles/14/107/2021/) dataset

##### The goal of this session is to experiment with using a deep neural network with the aim of trying and predict whether a given geographical location is experiencing an extreme weather event in the form of an atmospheric river or a tropical cyclone.

Import the necessary libraries

In [ ]:
import numpy as np
import matplotlib.pylab as plt
import xarray as xr
import torch
from torchsummary import summary
import tqdm

The first thing to do is to create a training and testing dataset.

 - If using Colab, you must have first added a shortcut to your GDrive - see [here](https://drive.google.com/drive/folders/1vrLE8nbpMHdTeWCpI2rrS1scQmQW_ujE?usp=share_link): 
 - If using your own machine, you must have downloaded the ClimateNet dataset from the above link and have it in the same directory as this notebook.

In [ ]:
ds = xr.open_dataset('/content/drive/MyDrive/climate_net/climatenet_data.nc')

View the dataset

In [ ]:
ds

Let us start by choosing some training inputs. You might want to do some data exploration to see what these they look like, and an idea of how they are related to each other. We also define our labels, which our network is trying to predict - 0 is no extreme event, 1 is an tropical cyclone and 2 is a atmospheric river.

In [ ]:
input_1 = ds.U850.data
input_2 = ds.V850.data
input_3 = ds.TMQ.data
input_4 = ds.PSL.data
labels = ds.LABELS.data

To make the problem more tractable, let's sparse down the data a little by filtering it.

*Note: is this the best way to filter the data? What happens to the performance of the model if we just sparse regularly, for example? These are questions you might come back to later. Convolutional networks are a special class of neural network that essentially optimise a filtering process to sequentially reduce dimensionaility whilst extracting relevant spatial information from images.*

In [ ]:
from scipy.ndimage.filters import gaussian_filter
filter_input_1 = gaussian_filter(input_1,sigma = [0,10,10])[:,::16,::16]
filter_input_2 = gaussian_filter(input_2,sigma = [0,10,10])[:,::16,::16]
filter_input_3 = gaussian_filter(input_3,sigma = [0,10,10])[:,::16,::16]
filter_input_4 = gaussian_filter(input_4,sigma = [0,10,10])[:,::16,::16]

print(filter_input_1.shape)

Now we have 48 x 72 = 3456 inputs per time step. We filter the labels slightly differently, i.e. a filtered cell is labelled as an extreme event according to its Gaussian centre cell.

In [ ]:
filter_labels = labels[:,::16,::16]

At this point the data is still in a nice form (i.e. we could do a contour plot for each timestep). Let us choose the first 67 time steps for training, and the remaining 16 for testing. We then need to convert it into the shape (training samples, number of inputs).

In [ ]:
X_train = np.stack((
        filter_input_1[:67].flatten(),
        filter_input_2[:67].flatten(),
        filter_input_3[:67].flatten(),
        filter_input_4[:67].flatten()
        ),
    axis=1
    )

X_test = np.stack((
        filter_input_1[67:].flatten(),
        filter_input_2[67:].flatten(),
        filter_input_3[67:].flatten(),
        filter_input_4[67:].flatten()
        ),
    axis=1
    )

print(X_train.shape)
print(X_test.shape)

We shall use one-hot encoded vectors for our labels, as to not imply any ordering in the data. This can be done using the identity matrix.

In [ ]:
def to_categorical(y, num_classes):
    """ 1-hot encodes a tensor """
    return np.eye(num_classes)[y]

We need to similarly flatten the labels into the shape (training samples, number of outputs)

In [ ]:
train_labels = filter_labels[:67].flatten()
test_labels  = filter_labels[67:].flatten()

print(train_labels.shape)
print(test_labels.shape)

nb_classes= 3
Y_train = to_categorical(train_labels, num_classes=nb_classes)
Y_test = to_categorical(test_labels  , num_classes=nb_classes)

print(Y_train.shape)
print(Y_test.shape)

The final step for preparing the training data is to normalise it. This is particularly important for our dataset since the range of values varies greatly for the different inputs (for example, PSL compared to TMQ). Here, we normalise by subtracting the mean and dividing by the standard deviation. Remember to normalize the test data by the same mean and standard deviation as the training data.

*You might want to experiment with different ways of normalizing the data, or look at what happens when there is no normalization.*

In [ ]:
mu    = np.mean(X_train, axis=0)
sigma = np.std( X_train, axis=0)

print(mu, sigma)

X_train = (X_train-mu)/sigma #Careful not to run this cell twice
X_test  = (X_test -mu)/sigma

To input the data to the network in batches we define a data generator. This is a function that returns a batch of data each time it is called. This is useful for large datasets that cannot fit into memory.

In [ ]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, data, labels):
        self.data   = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        data   = self.data[idx]
        labels = self.labels[idx]
        return data, labels

Convert our data to tensors and create a data generator for the training and testing data.

In [ ]:
X_train = torch.tensor(X_train, dtype=torch.float32)
Y_train = torch.tensor(Y_train, dtype=torch.float32)

val_split_percent = 5
val_split         = int(X_train.size(0)*(1-val_split_percent/100))

X_train, X_val = X_train[:val_split], X_train[val_split:]
Y_train, Y_val = Y_train[:val_split], Y_train[val_split:]

batch_size  = 128
train_steps = int(X_train.size(0)/batch_size)

train_dataloader = torch.utils.data.DataLoader(
    Dataset(X_train, Y_train),
    batch_size = batch_size,
    shuffle    = True
    )

val_dataloader = torch.utils.data.DataLoader(
    Dataset(X_val , Y_val),
    batch_size = batch_size,
    shuffle    = False
    )

Now that the data is prepared, let's build our first model. We will use a simple fully connected neural network with 2 hidden layers.

In [ ]:
class Model(torch.nn.Module):
    def __init__(self, input_size, nb_classes):
        super().__init__()

        self.net = torch.nn.Sequential(
            torch.nn.Linear(input_size, 200),
            torch.nn.ReLU(),
            torch.nn.Linear(200, 60),
            torch.nn.ReLU(),
            torch.nn.Linear(60, nb_classes), # classifying into 3 classes
        )

    def forward(self, x):
        return self.net(x)

# View model architecture and parameters
print(summary(Model(4, 3), (1, 4)))

We define a training loop, that trains the model for a given number of epochs. We also print the loss history at the end of each epoch.

In [ ]:
def fit(network,
        dataloader,
        val_dataloader,
        criterion,
        optimiser,
        epochs,
        train_steps):

    for epoch in range(epochs):  # loop over the dataset multiple times

        # progress bar for loss tracking
        with tqdm.trange(train_steps, ncols=100) as pbar:

            ### TRAINING LOOP ###

            running_loss = 0.0

            for steps, data in zip(pbar, dataloader):

                inputs, labels = data              # data is a list of (inputs, labels)

                outputs = network(inputs)          # forward pass

                loss = criterion(outputs, labels)  # compute loss

                loss.backward()                    # compute backward gradients

                optimiser.step()                   # update the parameters

                optimiser.zero_grad()              # zero the parameter gradients

                running_loss += loss.item()        # track loss

                if (steps+1)!=train_steps:         # if in training loop
                    # print loss tracker
                    pbar.set_postfix({
                        'epoch'      : f"{epoch+1}/{epochs}",
                        'loss'       : f"{running_loss/(steps+1):.3f}",
                        'val loss'   : f"-----",
                        })
                
                ### VALIDATION LOOP ###

                else:

                    val_steps        = 0
                    running_val_loss = 0.0
                    
                    for inputs, labels in val_dataloader:

                        # compute validation loss (do not track gradients)
                        with torch.no_grad(): loss = criterion(network(inputs), labels)

                        # track validation loss
                        running_val_loss += loss.item()
                        val_steps        += 1

                    # print loss tracker for validation set
                    pbar.set_postfix({
                        'epoch'      : f"{epoch+1}/{epochs}",
                        'loss'       : f"{running_loss    /(steps+1)    :.3f}",
                        'val loss'   : f"{running_val_loss/(val_steps+1):.3f}",
                        })
                    pbar.update()

            running_loss = 0.0

    return network

We instantiate the model and define our loss function and optimiser. As well as the number of epochs to train for.

In [ ]:
net       = Model(4, 3)
epochs    = 10
criterion = torch.nn.CrossEntropyLoss()
optimiser = torch.optim.Adam(net.parameters())

And train the model.

In [ ]:
net = fit(
    network        = net,
    dataloader     = train_dataloader,
    val_dataloader = val_dataloader,
    criterion      = criterion,
    optimiser      = optimiser,
    epochs         = epochs,
    train_steps    = train_steps
    )

We've trained our first model for making predictions on the data. Now let's have a look at how the model performs on the test set. We'll start by doing things qualitatively: reshaping the predicted classes back into their original spatio-temporal form and then plotting some time slices.

In [ ]:
X_test = torch.tensor(X_test, dtype=torch.float32)
Y_test = torch.tensor(Y_test, dtype=torch.float32)

with torch.no_grad():
    predicted_classes = net(X_test).numpy()
    predicted_labels = np.argmax(predicted_classes, axis=1).reshape((16, 48, 72))

In [ ]:
plt.figure(figsize=(15,15))
for timestep in range(5):
    plt.subplot(5,2,2*timestep+1)
    plt.pcolormesh(predicted_labels[timestep])
    if timestep==0:
        plt.title('Predictions')
    plt.subplot(5,2,2*timestep+2)
    plt.pcolormesh(np.reshape(test_labels, (16,48,72))[timestep])
    if timestep==0:
        plt.title('True labels')
plt.show()

At least qualititavely, our models seems to be doing an OK job at predicting some atmospheric rivers (in yellow) and a poor job at predicting tropical cyclones (in turquoise). Let's define and discuss some quantitative measures to expand on this. Firstly, accuracy: how often do predictions=labels?

In [ ]:
def accuracy(predicted, true):
    return np.sum(predicted==true)/predicted.shape[0]

In [ ]:
print(accuracy(predicted_labels.flatten(), test_labels.flatten()))

The model has very good accuracy on the test set. But how meaningful is this? Consider a baseline model that predicts class 0 for every grid cell. Then since most of the grid cells are indeed class 0, the accuracy will naturally be very high. Indeed, we can compute it:

In [ ]:
print(accuracy(np.zeros(test_labels.shape[0]), test_labels.flatten()))

Actually, our model only just exceeds the baseline by this metric. Ideally we would like a metric that captures how often the model is right when it predicts an extreme weather event. If we define a 'positive' to be a given class then we can define **precision** and **recall** as:

Precision = #True positives/(#True positives + #False positives)

Recall = #True positives/(#True positives + #False negatives)

Here a true positive means the model correctly predicted true, and a false positive means the model incorrectly predicted true. If we want to seek a balance between precision and recall we can use the so-called F1 score:

F1 = 2 x (precision x recall)/(precision + recall)

For an extended discussion on the meaning of these metrics, see this article: https://towardsdatascience.com/accuracy-precision-recall-or-f1-331fb37c5cb9

In [ ]:
def precision_label_n(predicted, true, label_n):
    tp = ((predicted==label_n).astype(int)*(true==label_n   ).astype(int)).sum()  #number of true positives
    fp = ((predicted==label_n).astype(int)*(1-(true==label_n).astype(int))).sum() #number of false positives
    return tp/(tp+fp)

In [ ]:
print('Cyclone precision', precision_label_n(predicted_labels.flatten(), test_labels.flatten(), label_n=1))
print('AR precision'     , precision_label_n(predicted_labels.flatten(), test_labels.flatten(), label_n=2))

In [ ]:
def recall_label_n(predicted, true, label_n):
    tp = ((predicted==label_n).astype(int)*(true==label_n     ).astype(int)).sum()  #number of true positives
    fn = ((1-(predicted==label_n).astype(int))*((true==label_n).astype(int))).sum() #number of false negatives
    return tp/(tp+fn)

In [ ]:
print('Cyclone recall'          , recall_label_n(predicted_labels.flatten(), test_labels.flatten(), label_n=1))
print('Atmospheric river recall', recall_label_n(predicted_labels.flatten(), test_labels.flatten(), label_n=2))

There is actually a pre-existing library function that will do this for you, for each label. The 'support' is just the number of (true) labels from a given class.

In [ ]:
from sklearn.metrics import precision_recall_fscore_support as score
precision, recall, fscore, support = score(test_labels.flatten(), predicted_labels.flatten())

In [ ]:
print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))
print('support: {}'.format(support))

Here we can clearly see that the fact that the number of instances of each class is seriously skewed is causing some problems for the model. In particular, the recall scores are particularly poor for extreme weather events, i.e. there are a lot of false negatives. This is bad news: we would probably rather have an overcautious model rather than our current very undercautious one (i.e. fails to predict a lot of true extreme weather events).

Of course, we should keep in mind that evaluating our model cell-by-cell is also probably not the best measure of performance: certainly the qualitative picture seems to suggest the regional performance on atmospheric rivers is actually OK: i.e. it gets the locations roughly correct (which cell-by-cell metrics of performance do not capture completely).

# The task
We have built a very basic model for predicting atmospheric weather events, and whilst it isn't great, there is an indication that it is picking up on some trends. By copying and adapting the code above, can you improve the model?

Some ideas to get you started:

1. You could try getting started by simply training the model for more epochs. How much does this improve the performance metrics?
2. We have an unbalanced dataset, what happens if you used a weighed loss function? You could also try using a different loss function.
3. Next, you might try changing the model design. Why not play around with the number of layers, the number of neurons per layer, and the activation functions?
4. What about the optimiser, is the generic Adam the best choice here, what about a larger/smaller learning rate, or a learning rate schedule?
5. You might try changing the inputs to the model (look at the data loading notebook from this morning for some ideas). You could also try adding more inputs to the model, changing the filtering procedure, and changing the normalization.
6. There is certainly spatial information in the original data that will be useful to a model for detecting extreme weather events. How might you harness this? One simple idea is to actually have latitude and/or longitude values of each grid cell each *as inputs to the model*.
7. Another simple idea is to reduce the size of the dataset by filtering out the high latitudes. This will reduce the skew in the number of labels for each class. 
8. A more complex idea is to create a dataset which predicts a cell label based on data values from that cell and the adjacent cells (note this will require substantially reshaping your training data).
9. What about trying a different type of model? You could try a convolutional neural network, or a vision transformer. You could also try a simpler type of ML model given the reasonably small dataset, such as a random forest or a support vector machine.

Do feel free to discuss with each other and us if you aren't sure what to do! We do stress that the aim of this notebook is not to create a ground-breaking model for predicting extreme weather events, but rather to get experience constructing a dataset, and playing around with simple machine learning models for making predictions.

### Choose inputs

In [ ]:
input_1 = ds.U850.data
input_2 = ds.V850.data
input_3 = ds.TMQ.data
input_4 = ds.PSL.data
# input_5 = ?
labels = ds.LABELS.data

### Filter to reduce number of data points

In [ ]:
from scipy.ndimage.filters import gaussian_filter
filter_input_1 = gaussian_filter(input_1,sigma = [0,10,10])[:,::16,::16]
filter_input_2 = gaussian_filter(input_2,sigma = [0,10,10])[:,::16,::16]
filter_input_3 = gaussian_filter(input_3,sigma = [0,10,10])[:,::16,::16]
filter_input_4 = gaussian_filter(input_4,sigma = [0,10,10])[:,::16,::16]
filter_labels = labels[:,::16,::16]

### Build training and testing datasets

In [ ]:
X_train = np.stack((
        filter_input_1[:67].flatten(),
        filter_input_2[:67].flatten(),
        filter_input_3[:67].flatten(),
        filter_input_4[:67].flatten()
        ),
    axis=1
    )

X_test = np.stack((
        filter_input_1[67:].flatten(),
        filter_input_2[67:].flatten(),
        filter_input_3[67:].flatten(),
        filter_input_4[67:].flatten()
        ),
    axis=1
    )

print(X_train.shape)
print(X_test.shape)

In [ ]:
train_labels = filter_labels[:67].flatten()
test_labels  = filter_labels[67:].flatten()

print(train_labels.shape)
print(test_labels.shape)

nb_classes= 3
Y_train = to_categorical(train_labels, num_classes=nb_classes)
Y_test = to_categorical(test_labels  , num_classes=nb_classes)

print(Y_train.shape)
print(Y_test.shape)

### Normalise

In [ ]:
mu    = np.mean(X_train, axis=0)
sigma = np.std( X_train, axis=0)

print(mu, sigma)

X_train = (X_train-mu)/sigma #Careful not to run this cell twice
X_test  = (X_test -mu)/sigma

In [ ]:
batch_size = 64

X_train = torch.tensor(X_train, dtype=torch.float32)
Y_train = torch.tensor(Y_train, dtype=torch.float32)

X_test = torch.tensor(X_test, dtype=torch.float32)
Y_test = torch.tensor(Y_test, dtype=torch.float32)

train_dataloader = torch.utils.data.DataLoader(
    Dataset(X_train, Y_train),
    batch_size = batch_size,
    shuffle    = True
    )

test_dataloader = torch.utils.data.DataLoader(
    Dataset(X_test , Y_test),
    batch_size = batch_size,
    shuffle    = False
    )

### Build model

In [ ]:
class Model(torch.nn.Module):
    def __init__(self, input_size, nb_classes):
        super().__init__()

        self.net = torch.nn.Sequential(
            torch.nn.Linear(input_size, 200),
            torch.nn.ReLU(),
            torch.nn.Linear(200, 60),
            torch.nn.ReLU(),
            torch.nn.Linear(60, nb_classes), # classifying into 3 classes
            torch.nn.Sigmoid()
        )

    def forward(self, x):
        return self.net(x)

# View model architecture and parameters
print(summary(Model(4, 3), (1, 4)))

In [ ]:
net       = Model(4, 3)
epochs    = 10
criterion = torch.nn.CrossEntropyLoss()
optimiser = torch.optim.Adam(net.parameters())

### Train model

In [ ]:
net = fit(
    network        = net,
    dataloader     = train_dataloader,
    val_dataloader = val_dataloader,
    criterion      = criterion,
    optimiser      = optimiser,
    epochs         = epochs,
    train_steps    = train_steps
    )

### Test model

In [ ]:
X_test = torch.tensor(X_test, dtype=torch.float32)
Y_test = torch.tensor(Y_test, dtype=torch.float32)

with torch.no_grad():
    predicted_classes = net(X_test).numpy()
    predicted_labels = np.argmax(predicted_classes, axis=1).reshape((16, 48, 72))

In [ ]:
plt.figure(figsize=(15,15))

for timestep in range(5):

    plt.subplot(5,2,2*timestep+1)
    plt.pcolormesh(predicted_labels[timestep])
    if timestep==0: plt.title('Predictions')

    plt.subplot(5,2,2*timestep+2)
    plt.pcolormesh(np.reshape(test_labels, (16,48,72))[timestep])
    if timestep==0: plt.title('True labels')

plt.show()

### Performance metrics

In [ ]:
from sklearn.metrics import precision_recall_fscore_support as score
precision, recall, fscore, support = score(test_labels.flatten(), predicted_labels.flatten())

In [ ]:
print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))
print('support: {}'.format(support))